In [41]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout,LSTM, GRU
from tensorflow.keras.preprocessing import sequence
import pandas as pd

In [42]:
# from google.colab import drive
# drive.mount('/content/drive')

In [43]:
def read_csv(filename):
  data_frame = pd.read_csv(filename)
  x = np.asarray(data_frame["sentence"])
  y = np.asarray(data_frame["label"], dtype = int)
  return x,y

In [44]:
x_train , y_train =read_csv('/content/drive/MyDrive/RNN_Networks/Emoji_Text_Classification/train.csv')
x_test , y_test = read_csv('/content/drive/MyDrive/RNN_Networks/Emoji_Text_Classification/test.csv')

In [45]:
def label_to_emoji(label):
  emojes = ['❤','⚽','😀','😒','🍴']
  return emojes[label]

In [46]:
index = 9
print(x_train[index], label_to_emoji(y_train[index]))

I want to go play ⚽


In [47]:
#convert label to one_hot
y_train_oh = tf.keras.utils.to_categorical(y_train ,5)
y_test_oh= tf.keras.utils.to_categorical(y_test ,5)

In [48]:
%cd /content/drive/MyDrive/RNN_Networks

/content/drive/MyDrive/RNN_Networks


In [49]:
# !wget nlp.stanford.edu/data/glove.6B.zip --no-verbose
# !unzip -q glove.6B.zip -d glove.6B

In [50]:
def read_glove_vectors(glove_file):
  f = open(glove_file, encoding = "utf8")

  words = set()
  words_to_vec = dict()
  for line in f:
      line = line.strip().split()
      word = line[0]
      vec = line[1:]
      words.add(word)
      words_to_vec[word] =np.array(vec, dtype = np.float64)
  return words_to_vec

In [51]:
words_to_vec = read_glove_vectors("./glove.6B/glove.6B.200d.txt")

In [52]:
def sentence_to_avg(sentence):
  words = sentence.lower().split() 
  sum_vectors = np.zeros((200,))
  for w in words:
    sum_vectors += words_to_vec[w]

  avg_vectors = sum_vectors / len(words)
  return avg_vectors

In [53]:
x_train_avg =[]
for i in range(x_train.shape[0]):
    x_train_avg.append(sentence_to_avg(x_train[i]))

x_train_avg = np.array(x_train_avg)

In [54]:
print(x_train_avg.shape)
print(y_train_oh.shape)

(132, 200)
(132, 5)


In [55]:
class EmojeNet_v1(Model):
  def __init__(self):
    super().__init__()
    self.dense =Dense(5, input_shape=(50,), activation = 'softmax')

  def call(self,x):
      x = self.dense(x)
      return(x)

In [56]:
model_v1 = EmojeNet_v1()

In [57]:
model_v1.compile(optimizer = tf.keras.optimizers.Adam(),
               loss = 'categorical_crossentropy',
               metrics = ['accuracy']  )

In [58]:
# model_v1.fit(x_train_avg,y_train_oh,epochs=400, shuffle=True)

In [59]:
x_test_avg =[]
for i in range(x_test.shape[0]):
    x_test_avg.append(sentence_to_avg(x_test[i]))

x_test_avg = np.array(x_test_avg)

In [60]:
model_v1.evaluate(x_test_avg,y_test_oh)

2/2 [==============================] - 0s 5ms/step - loss: 1.5731 - accuracy: 0.2679


[1.5730961561203003, 0.2678571343421936]

In [61]:
x_me = np.array(['i adore you','i love you','funny lol','lets play with a ball','food is ready','not feeling happy and funny'])
y_me =np.array([[0],[0],[2],[1],[4],[3]])
x_me_avg = []
for x in x_me:
  x_me_avg.append(sentence_to_avg(x))
x_me_avg = np.array(x_me_avg)

pred = model_v1.predict(x_me_avg)

for i in range(x_me.shape[0]):
    print(x_me[i], label_to_emoji(np.argmax(pred[i])))

1/1 [==============================] - 0s 32ms/step
i adore you 😒
i love you 😒
funny lol 🍴
lets play with a ball ⚽
food is ready 😒
not feeling happy and funny 😒


In [62]:
class EmojiNet_v2_Lstm(Model):
      def __init__(self):
          super().__init__()

          self.dense =Dense(5, activation = 'softmax')
          self.lstm_1 = LSTM(1024, return_sequences=True)
          self.dropuot_1=Dropout(0.5)
          self.lstm_2 = LSTM(512)
          self.dropuot_2=Dropout(0.5)

      def call(self,x):
          x=self.lstm_1(x)
          x=self.dropuot_1(x)
          x=self.lstm_2(x)
          x=self.dropuot_2(x)
          x = self.dense(x)
          return x

In [63]:
model_v2=EmojiNet_v2_Lstm()

In [64]:
model_v2.compile(optimizer = tf.keras.optimizers.Adam(lr = 0.0001),
               loss = 'categorical_crossentropy',
               metrics = ['accuracy']  )

In [65]:
def convert_sentenses_to_embeddings(x):
    emb_matrix=np.zeros((x.shape[0],  #number of data
                        10,#leng of longest sentence
                        200  # size of embedding vector
                        ))
    for i in range(x.shape[0]):
      words = x[i].lower().split()
      for j in range(len(words)):
        emb_matrix[i,j,:]= words_to_vec[words[j]]
    return emb_matrix    

In [66]:
x_me = np.array(['funny lol','lets play with a ball','food is ready'])
# print(convert_sentenses_to_embeddings(x_me))

In [67]:
x_train_embs=convert_sentenses_to_embeddings(x_train)
x_train_embs.shape

(132, 10, 200)

In [68]:

y_train = y_train.reshape(132,1)
print(y_train.shape)

(132, 1)


In [81]:
model_v2.fit(x_train_embs, y_train_oh, epochs=20, batch_size=16, shuffle=True)

Epoch 1/20
9/9 [==============================] - 5s 554ms/step - loss: 0.9237 - accuracy: 0.6818
Epoch 2/20
9/9 [==============================] - 5s 591ms/step - loss: 0.6919 - accuracy: 0.8258
Epoch 3/20
9/9 [==============================] - 5s 553ms/step - loss: 0.5745 - accuracy: 0.8182
Epoch 4/20
9/9 [==============================] - 5s 529ms/step - loss: 0.4813 - accuracy: 0.8561
Epoch 5/20
9/9 [==============================] - 5s 526ms/step - loss: 0.3304 - accuracy: 0.8864
Epoch 6/20
9/9 [==============================] - 5s 541ms/step - loss: 0.2424 - accuracy: 0.9242
Epoch 7/20
9/9 [==============================] - 5s 547ms/step - loss: 0.1495 - accuracy: 0.9545
Epoch 8/20
9/9 [==============================] - 5s 544ms/step - loss: 0.2303 - accuracy: 0.9318
Epoch 9/20
9/9 [==============================] - 5s 534ms/step - loss: 0.1802 - accuracy: 0.9394
Epoch 10/20
9/9 [==============================] - 5s 543ms/step - loss: 0.2347 - accuracy: 0.9015
Epoch 11/20
9/9 [==

In [82]:
x_test_embs = convert_sentenses_to_embeddings(x_test)
y_test_oh = tf.keras.utils.to_categorical(y_test, 5)
model_v2.evaluate(x_test_embs, y_test_oh)

2/2 [==============================] - 0s 126ms/step - loss: 0.4362 - accuracy: 0.8571


[0.4362258017063141, 0.8571428656578064]

In [83]:
x_example = np.array(["not sad", "i adore you", "i love you", "funny lol", "lets play with a ball", "food is ready", "not feeling happy"])
y_example = np.array([[2],[0],[0],[2],[1],[4],[3]])
x_example_embedd = convert_sentenses_to_embeddings(x_example)
pred = model_v2.predict(x_example_embedd)
for i in range(x_example.shape[0]):
    print(x_example[i], label_to_emoji(np.argmax(pred[i])))

1/1 [==============================] - 0s 95ms/step
not sad 😀
i adore you ❤
i love you ❤
funny lol 😀
lets play with a ball ⚽
food is ready 🍴
not feeling happy 😀


In [72]:
class EmojiNet_v3_Gru(Model):
      def __init__(self):
          super().__init__()

          self.dense =Dense(5, activation = 'softmax')
          self.gru_1 = GRU(1024, return_sequences=True)
          self.dropuot_1=Dropout(0.5)
          self.gru_2 = GRU(512)
          self.dropuot_2=Dropout(0.5)
      def call(self,x):
          x=self.gru_1(x)
          x=self.dropuot_1(x)
          x=self.gru_2(x)
          x=self.dropuot_2(x)
          x = self.dense(x)
          
          return x

In [73]:
model_v3=EmojiNet_v3_Gru()

In [74]:
model_v3.compile(optimizer = tf.keras.optimizers.Adam(lr = 0.0001),
               loss = 'categorical_crossentropy',
               metrics = ['accuracy']  )

In [75]:
model_v3.fit(x_train_embs, y_train_oh, epochs=20, batch_size=16, shuffle=True)

Epoch 1/20
9/9 [==============================] - 8s 386ms/step - loss: 1.5891 - accuracy: 0.2803
Epoch 2/20
9/9 [==============================] - 3s 338ms/step - loss: 1.5107 - accuracy: 0.3788
Epoch 3/20
9/9 [==============================] - 3s 344ms/step - loss: 1.4608 - accuracy: 0.3788
Epoch 4/20
9/9 [==============================] - 3s 388ms/step - loss: 1.3872 - accuracy: 0.5076
Epoch 5/20
9/9 [==============================] - 3s 334ms/step - loss: 1.2982 - accuracy: 0.6212
Epoch 6/20
9/9 [==============================] - 3s 321ms/step - loss: 1.1646 - accuracy: 0.6439
Epoch 7/20
9/9 [==============================] - 3s 332ms/step - loss: 1.0111 - accuracy: 0.7045
Epoch 8/20
9/9 [==============================] - 3s 326ms/step - loss: 0.8465 - accuracy: 0.7727
Epoch 9/20
9/9 [==============================] - 3s 337ms/step - loss: 0.6965 - accuracy: 0.7727
Epoch 10/20
9/9 [==============================] - 3s 329ms/step - loss: 0.5884 - accuracy: 0.8106
Epoch 11/20
9/9 [==

In [78]:
x_test_embs = convert_sentenses_to_embeddings(x_test)
y_test_oh = tf.keras.utils.to_categorical(y_test, 5)
model_v3.evaluate(x_test_embs, y_test_oh)

2/2 [==============================] - 0s 71ms/step - loss: 0.5678 - accuracy: 0.7679


[0.5678073763847351, 0.7678571343421936]

In [79]:
x_example = np.array(["not sad", "i adore you", "i love you", "funny lol", "lets play with a ball", "food is ready", "not feeling happy"])
y_example = np.array([[2],[0],[0],[2],[1],[4],[3]])
x_example_embedd = convert_sentenses_to_embeddings(x_example)
pred = model_v3.predict(x_example_embedd)
for i in range(x_example.shape[0]):
    print(x_example[i], label_to_emoji(np.argmax(pred[i])))

1/1 [==============================] - 0s 52ms/step
not sad 😀
i adore you ❤
i love you ❤
funny lol 😀
lets play with a ball ⚽
food is ready 🍴
not feeling happy 😀
